In [2]:
import pandas as pd
import yfinance as yf
import seaborn as sns
from datetime import date, timedelta
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import scipy.stats as stats
from statsmodels.tsa.stattools import adfuller
from datetime import date
d2 = date.today() - timedelta(days=200)
d2 = d2.strftime("%Y-%m-%d")
plt.rcParams['figure.figsize'] = [20, 10]

In [3]:
def lines():
    for y, color, linestyle in [(0, 'black', '-'),
                                (1.0, 'green', '-'),
                                (2.5, 'green', '--'),
                                (3, 'magenta', '--'),
                                (-1.0, 'red', '--'),
                                (-2.5, 'red', '--'),
                                (-3, 'magenta', '--')]:
        plt.axhline(y=y, color=color, linestyle=linestyle)

In [5]:
path =r"/mnt/d/Wall-Street-Sessions/ind_nifty500list.csv"
tickers=pd.read_csv(path)
tickers['Symbol']=tickers['Symbol']+str('.NS')
download=(tickers['Symbol'].to_list())

In [6]:
df=yf.download(tickers=download,start=d2,)['Close']

[*********************100%***********************]  499 of 499 completed


In [8]:
df.fillna(method='bfill',inplace=True)

/tmp/ipykernel_13392/1626352436.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill',inplace=True)


In [9]:
# Calculate percentage change
df2 = df.pct_change().dropna()

# Calculate correlation matrix
corr = df2.corr()

# Print the columns of the correlation DataFrame
print("Columns in correlation DataFrame:", corr.columns)
# Solution: Rename the index before resetting
corr.index.name = None  # Temporarily remove the index name
corr_long = corr.stack().reset_index()  # Now stack and reset index
corr_long.columns = ['stock1', 'stock2', 'correlation']  # Rename columns for clarity

# Optionally, drop duplicate pairs (if you want unique pairs)
corr_long = corr_long[corr_long['stock1'] < corr_long['stock2']]

# Display the long format DataFrame
print("\nLong Format Correlation DataFrame:")
print(corr_long)

Columns in correlation DataFrame: Index(['360ONE.NS', '3MINDIA.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS',
       'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ACC.NS', 'ACE.NS',
       ...
       'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS',
       'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZFCVINDIA.NS', 'ZOMATO.NS',
       'ZYDUSLIFE.NS'],
      dtype='object', name='Ticker', length=499)

Long Format Correlation DataFrame:
               stock1         stock2  correlation
1           360ONE.NS     3MINDIA.NS     0.018397
2           360ONE.NS    AARTIIND.NS     0.252426
3           360ONE.NS       AAVAS.NS     0.176271
4           360ONE.NS         ABB.NS     0.191642
5           360ONE.NS  ABBOTINDIA.NS     0.167776
...               ...            ...          ...
246508  ZENSARTECH.NS      ZOMATO.NS     0.228518
246509  ZENSARTECH.NS   ZYDUSLIFE.NS     0.287881
247006   ZFCVINDIA.NS      ZOMATO.NS     0.151472
247007   ZFCVINDIA.NS   ZYDUSLIFE.NS     0.071670
247505  

/tmp/ipykernel_13392/1434609956.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df2 = df.pct_change().dropna()


In [10]:
##The final dataframe without repetitions
Finale=corr_long.loc[corr_long['correlation']!=1]
Finale=Finale.sort_values(by='correlation', ascending=False)
Finale=Finale.drop_duplicates('correlation')

In [11]:
##Selecting stocks with atleast 0.5 corrcoef value

Finale=Finale.loc[Finale['correlation']>0.5]
Finale

,stock1,stock2,correlation
178174,PFC.NS,RECLTD.NS,0.943231
47281,CENTRALBK.NS,UCOBANK.NS,0.910906
116503,IOB.NS,UCOBANK.NS,0.903964
47045,CENTRALBK.NS,IOB.NS,0.891360
5990,ADANIENT.NS,ADANIPORTS.NS,0.888652
...,...,...,...
118350,IRCON.NS,MTARTECH.NS,0.500100
48969,CESC.NS,GAIL.NS,0.500057
61640,CUMMINSIND.NS,RBLBANK.NS,0.500039
161762,MTARTECH.NS,SCHNEIDER.NS,0.500027


In [24]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.tsa.stattools import adfuller

# Function to get stock data for a given stock from the DataFrame
def get_stock_data(stock_name, df):
    return np.array(df[stock_name])

# Function to calculate correlation between two stocks
def calculate_correlation(stock1_name, stock2_name, df2):
    return df2[stock1_name].corr(df2[stock2_name])

# Function to retrieve industry names for two stocks
def get_industry_names(stock1_name, stock2_name, tickers):
    industry1 = tickers.loc[tickers['Symbol'] == stock1_name].iloc[0]['Industry']
    industry2 = tickers.loc[tickers['Symbol'] == stock2_name].iloc[0]['Industry']
    return industry1, industry2

# Function to calculate residuals between two stocks
def calculate_residuals(stock1, stock2):
    return stock1 / stock2

# Function to determine Target Ratio and Stop Loss Ratio
def calculate_ratios(residuals, stock1, stock2):
    current_ratio = stock1[-1] / stock2[-1]
    mean_ratio = np.mean(residuals)
    std_ratio = np.std(residuals)

    if residuals[-1] > mean_ratio:
        target_ratio = mean_ratio + std_ratio
        sl_ratio = mean_ratio + 3 * std_ratio
    else:
        target_ratio = mean_ratio - std_ratio
        sl_ratio = mean_ratio - 3 * std_ratio

    return target_ratio, sl_ratio, current_ratio

# Function to perform ADF test on the residuals
def perform_adf_test(residuals):
    residuals_zscore = stats.zscore(residuals)  # Convert residuals to z-scores
    result = adfuller(residuals_zscore)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.3f}')
    
    adf_passed = result[1] < 0.05  # ADF test passed if p-value < 0.05
    return adf_passed, result[1]

# Main function to run pair trading logic for a stock pair
def PairTrade(i, Finale, df, df2, tickers):
    stock1_name = Finale['stock1'].iloc[i]
    stock2_name = Finale['stock2'].iloc[i]

    stock1 = get_stock_data(stock1_name, df)
    stock2 = get_stock_data(stock2_name, df)
    
    correlation = calculate_correlation(stock1_name, stock2_name, df2)
    
    industry1, industry2 = get_industry_names(stock1_name, stock2_name, tickers)
    
    residuals = calculate_residuals(stock1, stock2)
    target_ratio, sl_ratio, current_ratio = calculate_ratios(residuals, stock1, stock2)
    
    adf_passed, p_value = perform_adf_test(residuals)

    if adf_passed:
        print("ADF test passed")
        print("Target:", abs((target_ratio - current_ratio) / current_ratio))
    else:
        print("ADF test failed")
    
    return stock1_name, stock2_name, p_value, residuals[-1], correlation, industry1, industry2, current_ratio, sl_ratio, target_ratio

# Function to run pair trading for all pairs
def run_pair_trading(Finale, df, df2, tickers):
    Trades = []
    for i in range(len(Finale)):
        Trades.append(PairTrade(i, Finale, df, df2, tickers))
    return Trades   

In [32]:
Trades

[('PFC.NS',
  'RECLTD.NS',
  0.0033595405704679388,
  -0.5565554284877988,
  0.943230983030238,
  'Financial Services',
  'Financial Services',
  0.8691732528554397,
  0.7836547051117836,
  0.8536530580441198),
 ('CENTRALBK.NS',
  'UCOBANK.NS',
  0.44437631083570567,
  0.054868289334715506,
  0.9109062859842721,
  'Financial Services',
  'Financial Services',
  1.1751172676440824,
  1.2906914150614632,
  1.2122065398357822),
 ('IOB.NS',
  'UCOBANK.NS',
  0.0039231874982674116,
  -1.7407603277034738,
  0.9039641749012707,
  'Financial Services',
  'Financial Services',
  1.1538976841456259,
  1.1315599344268827,
  1.1670380887610847),
 ('CENTRALBK.NS',
  'IOB.NS',
  0.37497496492694077,
  0.9616359466130149,
  0.8913598981313656,
  'Financial Services',
  'Financial Services',
  1.0183894844317745,
  1.0784925916029413,
  1.0195206851161698),
 ('ADANIENT.NS',
  'ADANIPORTS.NS',
  0.4614661100720557,
  -0.7020719672118672,
  0.8886518077884321,
  'Metals & Mining',
  'Services',
  2.0896

In [30]:
DailyTrades=pd.DataFrame(Trades,columns=['stock1','stock2','pvalue','zscore','correlation','Sector1','Sector2','Current Ratio','SL Ratio','Target Ratio'])
##Here we only select the pairs that have a pvalue of 0.05 or lesser 
DailyTrades=DailyTrades.loc[DailyTrades['pvalue']<=0.05]

In [31]:
DailyTrades

,stock1,stock2,pvalue,zscore,correlation,Sector1,Sector2,Current Ratio,SL Ratio,Target Ratio
0,PFC.NS,RECLTD.NS,0.003360,-0.556555,0.943231,Financial Services,Financial Services,0.869173,0.783655,0.853653
2,IOB.NS,UCOBANK.NS,0.003923,-1.740760,0.903964,Financial Services,Financial Services,1.153898,1.131560,1.167038
7,IOB.NS,MAHABANK.NS,0.001096,2.194196,0.872927,Financial Services,Financial Services,1.033200,1.048906,1.009924
18,ABB.NS,SIEMENS.NS,0.012939,-0.528402,0.829011,Capital Goods,Capital Goods,1.116837,1.039334,1.102049
41,CANBK.NS,CENTRALBK.NS,0.000316,0.747940,0.775865,Financial Services,Financial Services,1.866375,1.968881,1.877848
59,ACC.NS,ADANIENT.NS,0.011168,-0.032730,0.761248,Construction Materials,Metals & Mining,0.802134,0.682876,0.763258
84,ADANIENT.NS,BANKBARODA.NS,0.037526,-0.926022,0.745052,Metals & Mining,Financial Services,11.562691,10.410854,11.521606
107,NMDC.NS,TATASTEEL.NS,0.028166,-0.365609,0.731556,Metals & Mining,Metals & Mining,1.448852,1.297985,1.412522
108,ADANIPORTS.NS,BEL.NS,0.043241,-0.188877,0.731352,Services,Capital Goods,4.991340,4.017952,4.710478


In [33]:
TheFinalTrades=DailyTrades.loc[(abs(DailyTrades['zscore'])>1)&(abs(DailyTrades['pvalue'])<0.05)]

In [34]:
TheFinalTrades

,stock1,stock2,pvalue,zscore,correlation,Sector1,Sector2,Current Ratio,SL Ratio,Target Ratio
2,IOB.NS,UCOBANK.NS,0.003923,-1.740760,0.903964,Financial Services,Financial Services,1.153898,1.131560,1.167038
7,IOB.NS,MAHABANK.NS,0.001096,2.194196,0.872927,Financial Services,Financial Services,1.033200,1.048906,1.009924
